In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse

#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1612273567887_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:


#Query some reviews
query_1 = "select distinct r.ugc_id,  mg.name, r.review_headline, r.review_comments, \
case when (contains(r.observations, 'CR') )  then 1 else 0 end as CR \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where r.locale in ('en_US','en_GB','en_CA') and contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and r.created_date > '2020-01-01' \
and mg.usage_type != 'Test' \
"

#and ugc_id in (315642957,276609384,273955886, 285059791)
#and contains(r.observations, 'CR') \
#and mg.name in ('MidwayUSA', 'Midsouth Shooters','Ulta') and ugc_id in (315642957,276609384,273955886, 285059791) \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8060403 entries, 0 to 8060402
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   UGC_ID           int64 
 1   NAME             object
 2   REVIEW_HEADLINE  object
 3   REVIEW_COMMENTS  object
 4   CR               int64 
dtypes: int64(2), object(3)
memory usage: 307.5+ MB

In [6]:
reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS']

reviews['NAME_1ST_PART_RAW'] = reviews['NAME'].apply(lambda x: x.split()[0])
reviews['NAME_1ST_PART'] = reviews.apply(lambda x: 'blick' if x['NAME_1ST_PART_RAW']=='Dick' else x['NAME_1ST_PART_RAW'], axis = 1)

#Lower case
reviews['ALL_TEXT'] = reviews['ALL_TEXT'].str.lower()
reviews['NAME_1ST_PART'] = reviews['NAME_1ST_PART'].str.lower()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                       NAME  ... NAME_1ST_PART_RAW NAME_1ST_PART
0  317499743             Russell Stover  ...           Russell       russell
1  317547841  Itzy Ritzy Seller Ratings  ...              Itzy          itzy
2  317632677              Bonobos, Inc.  ...          Bonobos,      bonobos,
3  317604630              Bonobos, Inc.  ...          Bonobos,      bonobos,
4  317669394  Itzy Ritzy Seller Ratings  ...              Itzy          itzy

[5 rows x 8 columns]

In [8]:
#Remove mentions if client name in review text
reviews['ALL_TEXT_2'] = reviews.apply(lambda x: str(x['ALL_TEXT']).replace(str(x['NAME_1ST_PART']),''), axis = 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Ignore where midwayusa client name bc target references don't refer to Target the retailer but target as in target practice
#Can't look for case sensitivity bc people still start a sentence with target and therefore it's capitalized
#Midsouth Shooters as well, Cheaper Than Dirt as well
#3Rivers Archery, JoeBobOutfitters - Terminated, Leupold - TERMINATED 
reviews['ALL_TEXT_3'] = reviews.apply(lambda x: str(x['ALL_TEXT_2']).replace('target','') if (x['NAME']=='MidwayUSA' or x['NAME']=='Midsouth Shooters' or x['NAME']=='Cheaper Than Dirt' or x['NAME']=='3Rivers Archery' or x['NAME']=='JoeBobOutfitters - Terminated' or x['NAME']=='Leupold - TERMINATED' ) else x['ALL_TEXT_2'], axis = 1)

#Ignore where Citi card holders mention Costco bc it's a Costco related credit card
reviews['ALL_TEXT_4'] = reviews.apply(lambda x: str(x['ALL_TEXT_3']).replace('costco','') if (x['NAME']=='Citigroup Inc.') else x['ALL_TEXT_3'], axis = 1)

#Ignore where sears in mentioned in BBQ Guys reviews, almost always refering to searing of meat
reviews['ALL_TEXT_5'] = reviews.apply(lambda x: str(x['ALL_TEXT_4']).replace('sears','') if (x['NAME']=='BBQ Guys') else x['ALL_TEXT_4'], axis = 1)

#Marshalls is a brand sold at Wickes, also at Guitar Center, and at Musician's Friend, Sam Ash
#Books-A-Million has series with Marshalls in the name
reviews['ALL_TEXT_6'] = reviews.apply(lambda x: str(x['ALL_TEXT_5']).replace('marshalls','') if (x['NAME']=='Wickes Building Supplies' or x['NAME']=='Guitar Center' or x['NAME']=="Musician's Friend" or x['NAME']=='Sam Ash' or x['NAME']=='Books-A-Million') else x['ALL_TEXT_5'], axis = 1)
reviews['ALL_TEXT_7'] = reviews.apply(lambda x: str(x['ALL_TEXT_6']).replace("marshall's",'') if (x['NAME']=='Wickes Building Supplies' or x['NAME']=='Guitar Center' or x['NAME']=="Musician's Friend" or x['NAME']=='Sam Ash' or x['NAME']=='Books-A-Million') else x['ALL_TEXT_6'], axis = 1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                       NAME  \
0  317499743  Russell Stover              
1  317547841  Itzy Ritzy Seller Ratings   
2  317632677  Bonobos, Inc.               
3  317604630  Bonobos, Inc.               
4  317669394  Itzy Ritzy Seller Ratings   

                                       REVIEW_HEADLINE  \
0  Less Nuts Every Year                                  
1  Great prices!                                         
2  Great everyday work pant                              
3  Great looking shirt!                                  
4  So far so good, we will see once I get my products.   

                                                                                                                                                                                                                                                                                                                                                                                                        

In [11]:

#Waitrose is owned by John Lewis - so shouldn't be marked for John Lewis - add to regex list or no?
#Remove john lewis, big lots
#Remove Goodwill - doesn't count as retailer
#If on or on- before target then ignore, lots of references to products being on target for size or whatever and not refering to retailer Target
#(?:(?<!ON[ -]))TARGET
#Ulta requested for Ipsy to be marked as retailer mention - Ipsy = subscription box that sends samples of products
reviews['regex_retailer'] = reviews['ALL_TEXT_7'].str.extractall(r'\b(AMAZON(?! (?:PAY|ALEXA|FIRE STICK))|E[ -]?BAY|OVERSTOCK|WAL[ -]?MART|(?:(?<!ON[ -]))TARGET|SEARS|ACE|LOWE\'?S|HOME DEPOT|MENARD\'?S|B ?& ?Q|WICKES|CABELA\'?S|FLEET FARM|DOLLAR TREE|DOLLAR GENERAL|FAMILY DOLLAR|SAM\'?S|COSTCO|MACY\'?S|NORDSTROM|JC[ ]?PENNE?Y|TJ[ -]?MAXX?|MARSHALL\'?S|(?:(?<!BOB ))ROSS|KOHL\'?S|J[ ]?CREW|CVS|WALGREEN\'?S|RITE[ -]?AID|QVC|HSN|KROGER|WHOLE FOODS|JEWEL OSCO|TJ\'?S|TRADER JOE\'?S?|WEGMAN\'?S|ALBERTSON|ALDI\'?S?|TESCO(?:(?! (?:CLUB[ ]?CARD|POINTS|VOUCHERS)))|CARREFOUR|SEPHORA|POTTERY BARN|IKEA|BED BATH & BEYOND|BED BATH AND BEYOND|WAYFAIR|ZAPPOS|FOOT LOCKER|DSW|AUTO[ ]?ZONE|ETSY|HOBBY LOBBY|MICHAEL\'?S|BLICK|DICK\'?S|REI|AETNA|STATE FARM|IPSY)\b', flags=re.I).unstack().apply(lambda x:','.join(x.dropna()), axis=1)
#.str.extract(r'([^@A-Z][-A-Z0-9:%_\+~#=]+\.(?:CO|COM|NET|ORG|GOV|EDU|APP|IN|BE|GL|NL|LY|FR)\b)', flags=re.I)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
reviews['retailer_not_client'] = pd.notnull(reviews['regex_retailer'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
reviews['model_CR'] = 0

reviews.loc[(reviews['retailer_not_client']==True ), 'model_CR'] = 1

#Consumer reports allows retailer mentions
reviews.loc[(reviews['NAME']=='Consumer Reports') | (reviews['NAME']=='Locally'), 'model_CR'] = 0


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
FP = reviews[(reviews['CR'] == 0) & (reviews['model_CR'] == 1)]
FN = reviews[(reviews['CR'] == 1) & (reviews['model_CR'] == 0)]
TP = reviews[(reviews['CR'] == 1) & (reviews['model_CR'] == 1)]
TN = reviews[(reviews['CR'] == 0) & (reviews['model_CR'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6012

In [16]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12334

In [17]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

29452

In [18]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8012605

In [19]:
FN.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         UGC_ID                NAME  \
1413  317678533  Crocs US             
2031  278836457  S&S Worldwide        
2295  278837532  West Marine          
2322  278839043  TSC                  
3374  271828671  Shutterfly           
3706  271828164  Big Lots             
4100  271769043  By Kilian            
5356  325556731  LUSH                 
6946  270551151  Stearns and Foster   
8276  316682211  Country Archer       

                                  REVIEW_HEADLINE  \
1413  Bad design, unlike the regular loafers        
2031  lies lies lies                                
2295  No                                            
2322  Disappointed                                  
3374  Perfect for gift for my 5 yr. old grandson!   
3706  I would buy another one it is really cheap    
4100  Just buy it...                                
5356  Very relaxing scent and color!                
6946  Low quality                                   
8276  Beef Jerky Sticks Review          

In [20]:
FN[['NAME','UGC_ID']].groupby(['NAME']).count().reset_index().sort_values(['UGC_ID'], ascending=False).head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                         NAME  UGC_ID
707  Ulta                      551   
318  John Lewis & Partners     391   
650  The Hartford - P&C        343   
743  Wickes Building Supplies  285   
33   Art.com - TERMINATED      274   
12   Advance Auto Parts        258   
126  Citigroup Inc.            255   
122  Cheaper Than Dirt         245   
170  Discount Tire             228   
633  TSC                       175

In [21]:
check=FN[FN['NAME']=='Ulta']
#check['trendmood']=check['ALL_TEXT_7'].str.extractall(r'\b(TRENDMOOD)\b',flags=re.I).unstack().apply(lambda x:','.join(x.dropna()), axis=1)
check.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           UGC_ID  NAME                             REVIEW_HEADLINE  \
43598   268525217  Ulta  The component was STOLEN from Lunar Beauty   
55126   324747859  Ulta  not worth the hype                           
66816   265135399  Ulta  LOVE                                         
154380  318236541  Ulta  LOVE                                         
168850  272525740  Ulta  The one I bought at Big lots is better.      

                                                                                                                                                                                                                                                                                                                                                                                                                                                 REVIEW_COMMENTS  \
43598   Component was stolen from Manny MUA's brand Lunar Beauty. Hate to see big companies stealing from indie brands.  

In [24]:
test2=check[check['trendmood']=='trendmood']
test2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UGC_ID                 13
NAME                   13
REVIEW_HEADLINE        13
REVIEW_COMMENTS        13
CR                     13
ALL_TEXT               13
NAME_1ST_PART_RAW      13
NAME_1ST_PART          13
ALL_TEXT_2             13
ALL_TEXT_3             13
ALL_TEXT_4             13
ALL_TEXT_5             13
ALL_TEXT_6             13
ALL_TEXT_7             13
regex_retailer         0 
retailer_not_client    13
model_CR               13
trendmood              13
dtype: int64

In [41]:
FP.groupby(['regex_retailer']).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                                UGC_ID  NAME  \
regex_retailer                                                                 
ace                                                             222     222    
ace,ace                                                         35      35     
ace,ace,ace                                                     2       2      
ace,ace,ace,ace                                                 1       1      
aetna                                                           16      16     
aetna,aetna                                                     1       1      
aetna,aetna,aetna                                               3       3      
aetna,aetna,aetna,aetna                                         1       1      
albertson                                                       10      10     
aldi                                                            67      67     
aldi's                                  

In [23]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("CR", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("NAME_1ST_PART", StringType(), True)\
,StructField("regex_retailer_insens_case", StringType(), True)\
,StructField("regex_retailer_sens_case", StringType(), True)\
,StructField("regex_retailer", StringType(), True)\
,StructField("retailer_not_client", StringType(), True)\
,StructField("model_CR", IntegerType(), True)])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FN = spark.createDataFrame(FN, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
FN.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","CR_FN_test3").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…